In [2]:
import sys 
import os 

sys.path.append('../..')

DATA_DIR = '../../data'
os.environ['DATA_DIR'] = DATA_DIR

from utils.data import load_data, get_all_vectorized
from utils.analyses.similarity.wordsim import calc_pair_sim
import numpy as np
import pandas as pd


from os.path import join as pjoin

ImportError: attempted relative import with no known parent package

# Predicting human similarity judgements

## Wrd Similarity and relatedness

In [ ]:
gpt_vec, cslb_vec, mc_vec, behv_sim = get_all_vectorized(vec='count')

In [3]:
wordsim = pd.read_csv(pjoin(DATA_DIR, '/wordsim353/combined.csv'))
wordsim = wordsim.rename(columns={'Word 1': 'word1', 'Word 2': 'word2', 'Human (mean)': 'rating'})

simlex = pd.read_csv(pjoin(DATA_DIR, 'simlex999/SimLex-999.txt'), sep='\t', usecols=['word1', 'word2', 'SimLex999'])
simlex = simlex.rename(columns={'SimLex999': 'rating'})

men = pd.read_csv(pjoin(DATA_DIR, 'men/MEN_dataset_natural_form_full'), sep=' ', names=['word1', 'word2', 'rating'], header=None)

yp = pd.read_csv(pjoin(DATA_DIR, 'yp/yp-130.csv'))
yp = yp.rename(columns={'similarity': 'rating'})

mturk771 = pd.read_csv(pjoin(DATA_DIR, 'mturk_771/mturk-771.csv'))
mturk771 = mturk771.rename(columns={'similarity': 'rating'})

mturk287 = pd.read_csv(pjoin(DATA_DIR, 'mturk_287/mturk-287.csv'))
mturk287 = mturk287.rename(columns={'similarity': 'rating'})

rw = pd.read_csv(pjoin(DATA_DIR, 'rw/rw.csv'))
rw = rw.rename(columns={'similarity': 'rating'})
                 

NameError: name 'pd' is not defined

In [ ]:
for dataset_name, wordsim_df in (('wordsim-353', wordsim), 
                                 ('simlex-999', simlex), 
                                 ('men', men),
                                 ('mturk-771', mturk771),
                                 ('mturk-287', mturk287),
                                 ('rw', rw),
                                 ('yp', yp)):
    ratings = []
    gpt_similarities = []
    cslb_similarities = []
    mc_similarities = []
    
    for row in wordsim_df.iterrows():
        word1 = row[1]['word1'].lower()
        word2 = row[1]['word2'].lower()
        rating = row[1]['rating']
        
        gpt_words = gpt_vec.index
        
        if word1 not in gpt_words:
            word1 = f'{word1}1'
            
        if word2 not in gpt_words:
            word2 = f'{word2}1'
        
       
        word1_in_gpt_cslb = word1 in gpt_words and word1 in cslb_vec.index and word1 in mc_vec.index
        word2_in_gpt_cslb = word2 in gpt_words and word2 in cslb_vec.index and word2 in mc_vec.index
        
        
        if word1_in_gpt_cslb and word2_in_gpt_cslb:
            ratings.append(rating)

            gpt_sim = calc_pair_sim(gpt_vec, word1, word2)
            gpt_similarities.append(gpt_sim)

            cslb_sim = calc_pair_sim(cslb_vec, word1, word2)
            cslb_similarities.append(cslb_sim)
            
            mc_sim = calc_pair_sim(mc_vec, word1, word2)
            mc_similarities.append(mc_sim)
    
    corr_cslb = np.corrcoef(cslb_similarities, ratings)[1][0]
    corr_gpt = np.corrcoef(gpt_similarities, ratings)[1][0]
    corr_mc = np.corrcoef(mc_similarities, ratings)[1][0]
    print(f'Dataset {dataset_name} has {len(ratings)} intersection word pairs and correlation of GPT-Dataset is {corr_gpt:.3} and CSLB-Dataset is {corr_cslb:.3}  Mc-Dataset is {corr_mc:.3}')
        
